# Model Deployment

## Importing the required Libraries

In [8]:
import joblib
import pandas as pd
import numpy as np
from flask import Flask, request, jsonify
import os
from dotenv import load_dotenv

# Loading the .env files
load_dotenv()

True

## Loading the model and the preprocessor

In [9]:
model_path = os.getenv('MODEL_PATH')
preprocessor_path = os.getenv('PREPROCESSOR_MODEL_PATH')

rf_classifier = joblib.load(model_path)
preprocessor = joblib.load(preprocessor_path)

## Creating the preprocessing function

In [10]:
def preprocess_new_data(new_data):
    # Convert new data to DataFrame
    df = pd.DataFrame(new_data)
    # Preprocess the data
    processed_data = preprocessor.transform(df)
    return processed_data

## Creating the prediction function

In [11]:
def make_prediction(new_data):
    # Preprocess the new data
    processed_data = preprocess_new_data(new_data)
    # Make predictions
    predictions = rf_classifier.predict(processed_data)
    # Print the predictions
    return predictions

## Flask API Setup

In [12]:
app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json(force=True)
    prediction = make_prediction(data)
    return jsonify({'prediction': prediction.tolist()})

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
g:\ML Portfolio\end-to-end machine learning project\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
127.0.0.1 - - [28/Jun/2024 15:07:12] "POST /predict HTTP/1.1" 200 -
